### Set Up

In [3]:
#pip install -r requirements.txt 

In [5]:
#pip install torch

In [1]:
import os

In [2]:
#FAQ dependency
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import random
import json
import torch
from dialogue_model.model import NeuralNet
from dialogue_model.nltk_utils import *

In [3]:
#Development dependency
import wolframalpha
import wikipedia #works as an encyclopidia and nothing else

import getpass
import logging

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [4]:
# WebDriver for selenium dependency
from webdriver_manager.chrome import ChromeDriverManager
chrome_driver = ChromeDriverManager().install()

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
stemmer = PorterStemmer()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### FAQ

In [8]:
class ChatBot:
    def __init__(self):
        self.bot_name = "PAR"
        self.model = None
        self.tags = None
        self.intents = None
        self.all_words = None

    def load_model(self):
        with open('dialogue_model/intents.json', 'r') as json_data:
            self.intents = json.load(json_data)

        FILE = "dialogue_model/data.pth"
        data = torch.load(FILE)

        input_size = data["input_size"]
        hidden_size = data["hidden_size"]
        output_size = data["output_size"]
        self.all_words = data['all_words']
        self.tags = data['tags']
        model_state = data["model_state"]

        self.model = NeuralNet(input_size, hidden_size, output_size).to(device)
        self.model.load_state_dict(model_state)
        self.model.eval()

    def get_response(self, sentence):
        sentence = tokenize(sentence)
        X = bag_of_words(sentence, self.all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = self.model(X)
        _, predicted = torch.max(output, dim=1)

        tag = self.tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        if prob.item() > 0.75:
            for intent in self.intents['intents']:
                if tag == intent["tag"]:
                    return random.choice(intent['responses'])
        else:
            return "I do not understand..."

    def run(self):
        self.load_model()
        print("Let's chat! (type 'quit' to exit)")
        while True:
            sentence = input("You: ")
            if sentence == "quit":
                break

            response = self.get_response(sentence)
            print(f"{self.bot_name}: {response}")

In [9]:
# # Create an instance of the ChatBot class and run it
bot = ChatBot()

# searching using wolframalpha, wikipedia and google search

In [10]:
class Search:
    def __init__(self):
        self.driver = None
        
    def search_wolfram(self,statement):
        self.driver = webdriver.Chrome(service=Service(chrome_driver))
        self.driver.get("https://www.wolframalpha.com/")
        search_box = self.driver.find_element(By.CLASS_NAME,"_i5f3")
        search_box.send_keys(statement)
        search_box.send_keys(Keys.RETURN)

    
    def search_wikipedia(self,statement):
        try:
            result = wikipedia.summary(statement)
            return result
        except:
            return "not"
    
    def search_google(self,statement):
        self.driver = webdriver.Chrome(service=Service(chrome_driver))
        self.driver.get(f"https://www.google.com/search?q={statement.replace(' ','+')}")

In [11]:
search = Search()

# Linkedin

In [12]:
class LinkedIN:
    def __init__(self,linkedin_mail,linkedin_password):
        self.driver = webdriver.Chrome(service=Service(chrome_driver))
        self.driver.get("https://www.linkedin.com/")

        # Wait for the page to load
        time.sleep(2)

        # Find the login form and enter your credentials
        email_input = self.driver.find_element(By.ID,"session_key")
        password_input = self.driver.find_element(By.ID,"session_password")
        email_input.send_keys(linkedin_mail)  # Replace with your email
        password_input.send_keys(linkedin_password)  # Replace with your password
        password_input.send_keys(Keys.RETURN)

        # Wait for the login to complete
        time.sleep(2)
        
    def find_person(self,person):
        #click search icon
        
        search_button = self.driver.find_element(By.XPATH,"//button[starts-with(@class,'search-global-typeahead')]")
        self.driver.execute_script("arguments[0].click();",search_button)
        search = self.driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")
        search.send_keys(person)
        search.send_keys(Keys.RETURN)  

    def message_person_searched(self,message):
        self.driver.find_element(By.CLASS_NAME,'entry-point').click()
        message_button = self.driver.find_element(By.XPATH,"//div[starts-with(@class,'msg-form__contenteditable')]")
        self.driver.execute_script("arguments[0].click();",message_button)
        p = self.driver.find_element(By.XPATH,"//div[starts-with(@class,'msg-form__contenteditable')]/child::p")
        p.send_keys(message)
        #p.send_keys(Keys.RETURN)
        time.sleep(5)
        submit_button = self.driver.find_element(By.XPATH,"//button[starts-with(@class,'msg-form__send-button')]")
        self.driver.execute_script("arguments[0].click();",submit_button)
        
        
        
def send_msg_linkedin():
    linkedin_mail = input("Enter linkedin mail id: ")
    linkedin_password = getpass.getpass('Write your password here: ')
    linkedin = LinkedIN(linkedin_mail,linkedin_password)
    person = input("enter the person name to search: ")
    linkedin.find_person(person)
    message = input("Enter message to send: ")
    linkedin.message_person_searched(message)
    print("message send successfully")

## Gmail creation and send

In [13]:
class EmailSender:
    def __init__(self, smtp_server, smtp_port, smtp_username, smtp_password):
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.smtp_username = smtp_username
        self.smtp_password = smtp_password

    def send_email(self, from_email, to_email, subject, body):
        # Create MIMEMultipart object
        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = to_email
        msg['Subject'] = subject

        # Add email body as plain text
        msg.attach(MIMEText(body, 'plain'))

        # Create SMTP connection
        smtp_conn = smtplib.SMTP(self.smtp_server, self.smtp_port)
        smtp_conn.ehlo()
        smtp_conn.starttls()
        smtp_conn.login(self.smtp_username, self.smtp_password)

        # Send email
        smtp_conn.sendmail(from_email, to_email, msg.as_string())

        # Close SMTP connection
        smtp_conn.quit()

        
def send_mail():
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587 
    smtp_username = input("enter mail id. Make sure less secure app access is on in this id: ")  # Your email address
    smtp_password = getpass.getpass('Write your password here:')  #Your email password

    from_email = smtp_username  # Sender email address
    to_email = input('tell recipient mail. Seperate by comma if many')  # Recipient email address
    subject = input('type subject of mail')  # Email subject
    body = input('type body of mail')  # Email body

    email_sender = EmailSender(smtp_server, smtp_port, smtp_username, smtp_password)
    email_sender.send_email(from_email, to_email, subject, body)

    print('mail sent successfully')

### Chat Face

In [14]:
def query(config):
    if config == "FAQ":
        bot.run()
    elif config == "Ask a mathematical question":
        statement = input("Provide the equation / question:- ")
        search.search_wolfram(statement)
    elif config == "Mail":
        send_mail()
    elif config == "Search for a person in Linkedin":
        send_msg_linkedin()
    elif config == "search online":
        statement = input("Provide your query:- ")
        result = search.search_wikipedia(statement)
        if result == "not":
            search.search_google(statement)
        else:
            print('Found a result in local wikipedia')
            print(result)
        

In [ ]:
import tkinter as tk

def on_option_selected(*args):
    # This function will be called every time the selected option changes
    config = var.get()
    print("Selected option:", config)
    time.sleep(1)
    window.destroy()  # Close the window and exit the code
    try:
        query(config)
    except Exception as e:
        # Catch the error and print the error message
        print("An error occurred:", e)



def on_window_close():
    global running
    running = False
    window.destroy()


running = True
while running:
    # Create a window and a variable to store the selected option
    window = tk.Tk()
    var = tk.StringVar()

    # Set the initial value of the variable and bind the on_option_selected function
    var.set("FAQ")
    var.trace("w", on_option_selected)

    # Create a dropdown menu with the options
    option_menu = tk.OptionMenu(window, var, "FAQ", "Ask a mathematical question", "Mail","Search for a person in Linkedin", "search online")
    option_menu.pack()

    # Bind the WM_DELETE_WINDOW event to the window object
    window.protocol("WM_DELETE_WINDOW", on_window_close)

    # Start the main loop
    
    window.mainloop()
